In [44]:
import environment.job_search_environment as job_search_env
import argparse
import os
import random
from supersuit import pad_observations_v0, pad_action_space_v0

import ray
from ray import tune
from ray.tune.registry import register_env

In [39]:
# NOTE: This is working on the Zoo, but not on my local machine (M1 compatibility issues)
from ray.rllib.algorithms.ppo import PPOConfig

In [40]:
from ray import tune
from ray.rllib.models.modelv2 import restore_original_dimensions
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.tune.registry import register_env
import torch
from torch import nn
from ray.rllib.utils.framework import try_import_tf, try_import_torch

In [41]:
tf1, tf, tfv = try_import_tf()
torch, _ = try_import_torch()

In [42]:
def env_creator(args):
    env = job_search_env.env()
    return env

In [47]:
job_search_env.env().observation_space(agent="candidate_0")

Dict('candidate_obs': Dict('accepted_offer': Dict('employer_0': Discrete(101)), 'counter_offers': Dict('employer_0': Tuple(Discrete(101), Discrete(11))), 'current_offers': Dict('employer_0': Tuple(Discrete(101), Discrete(11))), 'job_openings': Dict('employer_0': Discrete(2)), 'rejected_offers': Dict('employer_0': Tuple(Discrete(2), Discrete(101)))), 'employer_obs': Dict('accepted_offers': Dict('candidate_0': Discrete(2)), 'candidate_strengths': Dict('candidate_0': Discrete(101)), 'counter_offers': Dict('candidate_0': Tuple(Discrete(101), Discrete(11))), 'declined_offers': Dict('candidate_0': Tuple(Discrete(2), Discrete(101))), 'job_applicants': Dict('candidate_0': Discrete(2)), 'outstanding_offers': Dict('candidate_0': Tuple(Discrete(101), Discrete(11))), 'rejected_offers': Dict('candidate_0': Tuple(Discrete(2), Discrete(101))), 'remaining_budget': Discrete(101)))

In [19]:
env_name = "job_search"
register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

In [20]:
ray.init()

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [22]:
"""
In order to deal with the Dictionary space, need to pass a custom model to RLlib.
See: https://medium.com/@nima.siboni/rllib-with-dictionary-state-baa06b64470f
"""
class CandidateModelV0(TorchModelV2, nn.Module):
    def __init__(self, obs_space, act_space, num_outputs, *args, **kwargs):
        TorchModelV2.__init__(self, obs_space, act_space, num_outputs, *args, **kwargs)
        nn.Module.__init__(self)
#         self.layer1 = torch.cat()
        self.next_layers = nn.Sequential(
            nn.Linear(3136, 512),
            nn.ReLU(),
        )
        
        self.policy_fn = nn.Linear(512, num_outputs)
        self.value_fn = nn.Linear(512, 1)
        
    def forward(self, input_dict, state, seq_lens):
        model_out = self.model(input_dict["obs"])

In [23]:
ModelCatalog.register_custom_model("CandidateModelV0", CandidateModelV0)

In [24]:
# TODO: use policy_map to map different policies to candidate and employer agents

In [30]:
config = (
    PPOConfig()
    .environment(env=env_name, clip_actions=True)
    .debugging(log_level="ERROR")
    .framework(framework="torch")
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
).to_dict()

In [31]:
print (config)
config["observation_space"] = None
config["action_space"] = None


{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': 'job_search', 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': True, 'disable_env_checking': False, 'num_workers': 2, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector'>, 'sample_async': Fal

In [27]:
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 5000000},
    checkpoint_freq=10,
    local_dir="~/ray_results/" + env_name,
    config=config,
)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3875903284.py, line 1)

# Run 1
Got the error: 
```
AssertionError: Observation spaces for all agents must be identical. Perhaps SuperSuit's pad_observations wrapper can help (useage: `supersuit.aec_wrappers.pad_observations(env)`
```

# Run 2

```
AssertionError: homogenization only supports Discrete and Box spaces
```

# Run 3

Same error
```
Traceback (most recent call last):
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=1879526, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 139, in __init__
    self.add_workers(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 490, in add_workers
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 620, in foreach_worker
    remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=1879573, ip=128.36.108.57, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fa145f35d50>)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 492, in __init__
    self.env = env_creator(copy.deepcopy(self.env_context))
  File "/tmp/ipykernel_1861406/2067092441.py", line 2, in <lambda>
  File "/tmp/ipykernel_1861406/841127696.py", line 3, in env_creator
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/supersuit/multiagent_wrappers/padding_wrappers.py", line 33, in pad_observations_v0
    homogenize_ops.check_homogenize_spaces(spaces)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/supersuit/utils/action_transforms/homogenize_ops.py", line 30, in check_homogenize_spaces
    assert False, "homogenization only supports Discrete and Box spaces"
AssertionError: homogenization only supports Discrete and Box spaces

During handling of the above exception, another exception occurred:

ray::PPO.__init__() (pid=1879526, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 549, in setup
    raise e.args[0].args[2]
AssertionError: homogenization only supports Discrete and Box spaces
```

i.e. I cannot use the SuperSuit wrapper to fix the issue of observation spaces for all agents needing to be identical.

# Run 4

Fixed the issue by making all observation spaces and action spaces the same for all agents.

New issue:

```
Failure # 1 (occurred at 2022-12-12_17-37-17)
Traceback (most recent call last):
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=499392, ip=128.36.232.21, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 139, in __init__
    self.add_workers(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 490, in add_workers
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 620, in foreach_worker
    remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=499452, ip=128.36.232.21, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f8059b28df0>)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 567, in __init__
    self.policy_dict = _determine_spaces_for_multi_agent_dict(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 2121, in _determine_spaces_for_multi_agent_dict
    raise ValueError(
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!

During handling of the above exception, another exception occurred:

ray::PPO.__init__() (pid=499392, ip=128.36.232.21, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 549, in setup
    raise e.args[0].args[2]
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!
```